# C1 - Content Based Image Retrieval
### Team 8

In [12]:
import numpy as np
import cv2
import glob
import tqdm
import pickle
import os
import math
import matplotlib.pyplot as plt

In [21]:
class DataLoader():
    def __init__(self, folder_path):
        self.folder_path = folder_path

    def get_mask(self, img, mode, threshold_area=71000):
        
        if mode == 'rgb':
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        elif mode == 'hsv':
            gray = img[:,:,2].copy()
        else:
            gray = img[:,:,0].copy()
        blur = cv2.GaussianBlur(gray, (13,13), 0)
        thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2) # threshold based on local pixel neighborhood (11x11 block size)

        # Two pass dilate with horizontal and vertical kernel
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,5))
        dilate = cv2.dilate(thresh, horizontal_kernel, iterations=2)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,1))
        dilate = cv2.dilate(dilate, vertical_kernel, iterations=2)

        # Find contours, filter using contour threshold area, and draw rectangle
        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        '''
        cv2.RETR_EXTERNAL retrieves only the extreme outer contours.
        cv2.CHAIN_APPROX_SIMPLE compresses horizontal, vertical, and diagonal segments and leaves only their end points.
        Follows Satoshi Suzuki's algorithm:
            1- The algorithm works by border following, which means it traces the boundary of connected components in the image.
            2- It starts from the top-left corner of the image and looks for the first white pixel. Once found, it begins to follow the contour border.
            3- While following the contour, the algorithm keeps track of the direction in which it is moving to ensure it stays on the boundary.
            4- Once the entire contour is followed and the start point is reached again, the algorithm continues scanning the image for the next contour.
        '''

        mask = np.zeros(gray.shape, dtype=np.uint8)
        counter = 0
        areas = []
        coordinates = []
        for c in cnts:
            area = cv2.contourArea(c) # shoelace formula for convex shapes
            if area > threshold_area:
                x,y,w,h = cv2.boundingRect(c) # get bounding box
                areas.append((area, (x,y,w,h)))
                counter += 1

        # Sort areas and positions by area
        areas = sorted(areas, key=lambda x: x[0], reverse=True)

        for i in range(min(len(areas), 2)):
            x,y,w,h = areas[i][1]
            coordinates.append((x,y,w,h))
            mask[y:y+h, x:x+w] = 255 # draw bounding box on mask

                # counter += 1
                # if counter > 2:
                #     print('Error! Too many paintings in this image!')
                #     plt.imshow(img)
                #     plt.show()
                #     plt.imshow(mask, cmap='gray')
                #     plt.show()

        if counter == 0:
            print('Error! No paintings in this image!')
            plt.imshow(img)
            plt.show()
            plt.imshow(mask, cmap='gray')
            plt.show()

        return mask, coordinates
    
    def get_mask_text(self, img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
        opening = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)

        x = closing-opening
        x = (x>150).astype(np.uint8)

        kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (13,13))
        dilated = cv2.dilate(x, kernel2, iterations=2)

        ctns = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        areas = []
        for c in ctns[0]:
            x,y,w,h = cv2.boundingRect(c)
            if w > h:
                areas.append((cv2.contourArea(c), (x,y,w,h)))


        areas = sorted(areas, key=lambda x: x[0], reverse=True)
        x, y, w, h = areas[0][1]

        for _, shape in areas:
            if y > shape[1]-10 and y < shape[1]+10:
                if shape[0] < x:
                    w = (x+w) - shape[0]
                    x = shape[0]
                else:
                    w = (shape[0]+shape[2]) - x
        
        return [x,y,x+w,y+h]

    def create_blocks_array(self, image, blockNumber):
    
        # Set number of slices per axis
        axisSlice = int(math.sqrt(blockNumber))

        blocksArray = []
        # Split the image into vertical blocks
        split_h = np.array_split(image, axisSlice, axis = 0)
        
        for i in range(axisSlice):
            for j in range(axisSlice):
                # Split vertical blocks into square blocks
                split_hv = np.array_split(split_h[i], axisSlice, axis = 1)
                blocksArray.append(split_hv[j])
        return blocksArray

    def create_histogram(self, block, mask, d_hist, mode, bins):
        channels = cv2.split(block)

        if mode == 'lab' and d_hist < 3: channels = channels[1:]
        
        range_a, range_b = 256, 256
        if mode == 'hsv': range_a = 180

        if d_hist == 1:
            if mask is None:
                # Compute 1D histograms for each channel separately
                hist = [cv2.calcHist([chan], [0], None, [bins], [0, range_a if i == 0 else range_b]) for i,chan in enumerate(channels)]
            else:
                # Compute 1D histograms for each channel separately
                hist = [cv2.calcHist([chan[mask!=0]], [0], None, [bins], [0, range_a if i == 0 else range_b]) for i,chan in enumerate(channels)]

        elif d_hist == 2:
            if mask is None:
                # Compute 2D joint histograms for each pair of channels
                hist = [cv2.calcHist([channels[i], channels[j]], [0, 1], None, [bins, bins], [0, range_a if i == 0 else range_b, 0, range_b])
                            for i in range(len(channels)) for j in range(i+1, len(channels))]
            else:
                # Compute 2D joint histograms for each pair of channels
                hist = [cv2.calcHist([channels[i][mask!=0], channels[j][mask!=0]], [0, 1], None, [bins, bins], [0, range_a if i == 0 else range_b, 0, range_b])
                            for i in range(len(channels)) for j in range(i+1, len(channels))]

        else:
            if mask is None:
                # Compute 3D joint histogram for all three channels
                hist, _ = np.histogramdd([c.flatten() for c in channels], bins=(bins, bins, bins), range=[(0, range_a), (0, range_b), (0, range_b)])
            else:
                # Compute 3D joint histogram for all three channels
                hist, _ = np.histogramdd([c[mask != 0] for c in channels], bins=(bins, bins, bins), range=[(0, range_a), (0, range_b), (0, range_b)])

        return hist
    
    def get_features_by_blocks(self, image, level, mode, d_hist, bins, mask_text=None):

        # Get blocks using multi-level resolution
        blocksArray = []
        for lvl in range(level+1):
            for b in self.create_blocks_array(image, (2**lvl)*(2**lvl)):
                blocksArray.append(b)

        histograms = []
        # if mask is not None:
        #     if mask_text is not None:
        #         mask[mask_text[1]:mask_text[3], mask_text[0]:mask_text[2]] = 0

        #     blocksMasks = []
        #     for lvl in range(level+1):
        #         for b in self.create_blocks_array(mask, (2**lvl)*(2**lvl)):
        #             blocksMasks.append(b) 
            
        #     for block, mask_i in zip(blocksArray, blocksMasks):
        #         # Compute the histogram of the channel and append it to the list
        #         hist = self.create_histogram(block, mask_i, d_hist, mode, bins)
        #         if isinstance(hist, list):
        #             for h in hist:
        #                 histograms.append(h.flatten() / (block.shape[0]*block.shape[1]))
        #         else:
        #             histograms.append(hist.flatten() / (block.shape[0]*block.shape[1]))
        
        # else:
        for block in blocksArray:
            # Compute the histogram of the channel and append it to the list
            hist = self.create_histogram(block, None, d_hist, mode, bins)
            if isinstance(hist, list):
                for h in hist:
                    histograms.append(h.flatten() / (block.shape[0]*block.shape[1]))
            else:
                histograms.append(hist.flatten()  / (block.shape[0]*block.shape[1]))
            
        # Concatenate all histograms into a single feature vector
        return np.concatenate(histograms)
    
    def load_data(self, level = 3, d_hist = 1, bins = 64, remove_background=False, remove_text=False):
        # Get a list of all image file names in the folder
        image_files = glob.glob(self.folder_path+'/*.jpg')

        # Initialize an empty list to store the processed images and masks
        processed_features_rgb = dict()
        processed_features_hsv = dict()
        processed_features_lab = dict()
        masks, masks_text = [], []

        # Iterate over each image file
        for f in tqdm.tqdm(image_files):

            img_id = int(f.split('\\')[-1].split('.')[0].split('_')[-1])

            # Load the image
            image = cv2.imread(f)
            # Convert the image from BGR to lab color space
            image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
            # Convert the image from BGR to HSV color space
            image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

            # Remove background (it can be 2 paintings in the same image)
            if remove_background:
                mask_image, coordinates = self.get_mask(image, 'rgb')
               
                if remove_text:
                    coordinates = sorted(coordinates, key=lambda x: (x[0], x[1]))
                    masks_text_i = [[None]]*len(coordinates)
                    for i, (x,y,w,h) in enumerate(coordinates):
                        x_text, y_text, w_text, h_text = self.get_mask_text(image[y:y+h, x:x+w])
                        masks_text_i[i] = [x+x_text, y+y_text, x+x_text+w_text, y+y_text+h_text]
                    masks_text.append(masks_text_i)
                else:
                    for i in range(len(coordinates)):
                        masks_text.append([None])

            else:
                mask_image = None

                coordinates = [[0,0,image.shape[1],image.shape[0]]]
                if remove_text:
                    masks_text.append([self.get_mask_text(image)])
                else:
                    masks_text.append([None])

            masks.append(mask_image)
            
            features, features_hsv, features_lab = [], [], []
            for i in range(len(coordinates)):
                x,y,w,h = coordinates[i]

                relative_mask_text = None
                if masks_text[-1][i] is not None:
                    relative_mask_text = [masks_text[-1][i][0]-x, masks_text[-1][i][1]-y, masks_text[-1][i][2]-x, masks_text[-1][i][3]-y]

                # Get the features of the image
                f = self.get_features_by_blocks(image[y:y+h, x:x+w], level, 'rgb', d_hist, bins, mask_text=relative_mask_text)
                f_hsv = self.get_features_by_blocks(image_hsv[y:y+h, x:x+w], level, 'hsv', d_hist, bins, mask_text=relative_mask_text)
                f_lab = self.get_features_by_blocks(image_lab[y:y+h, x:x+w], level, 'lab', d_hist, bins, mask_text=relative_mask_text)

                features.append(f)
                features_hsv.append(f_hsv)
                features_lab.append(f_lab)

            # Append the features to the dict
            processed_features_rgb[img_id] = features
            processed_features_hsv[img_id] = features_hsv
            processed_features_lab[img_id] = features_lab

        return processed_features_rgb, processed_features_hsv, processed_features_lab, masks, masks_text
        

In [51]:
# Copied from https://github.com/benhamner/Metrics -> Metrics.Python.ml_metrics.average_precision.py
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

# Copied from https://github.com/benhamner/Metrics -> Metrics.Python.ml_metrics.average_precision.py
def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    result = []
    for a,p in zip(actual, predicted):
        for a_i, p_i in zip(a,p):
            result.append(apk([a_i],p_i,k))
    return np.mean(result)

# compute the histogram intersection between two feature vectors
def histogram_intersection(hist1, hist2):
    return np.sum(np.minimum(hist1, hist2))

# compute the euclidian distance between two feature vectors
def euclidian_distance(hist1, hist2):
    return np.sqrt(np.sum(np.square(hist1 - hist2)))

# compute the chi-squared distance between two feature vectors
def chi_squared_distance(hist1, hist2):
    return np.sum(np.square(hist1 - hist2) / (hist1 + hist2 + 1e-10))

# compute the bhattacharyya distance between two feature vectors
def bhattacharyya_distance(hist1, hist2):
    # Ensure that both histograms have the same shape
    assert hist1.shape == hist2.shape, "Histograms must have the same shape"
    # Calculate the Bhattacharyya coefficient
    bhattacharyya_coeff = np.sum(np.sqrt(hist1 * hist2))
    # Calculate the Bhattacharyya distance
    bhattacharyya_distance = -np.log(bhattacharyya_coeff)
    return bhattacharyya_distance

# compute the Helling distance (Hellinger kernel) between two feature vectors
def hellinger_kernel(hist1, hist2):
    return np.sum(np.sqrt(hist1*hist2))

def compare_images(query_features, bbdd_features, k, sim_func):
    result = []
    for id1,f1 in query_features.items():
        result_i = []
        for i,f_i in enumerate(f1):
            distances = []
            for id2,f2 in bbdd_features.items():
                distances.append((id2, sim_func(f_i,f2)))
                #get k smallest values from distances
            
            if sim_func in [euclidian_distance, chi_squared_distance, bhattacharyya_distance]:
                k_smallest = sorted(distances, reverse=False, key=lambda x: x[1])[:k]
            else:
                k_smallest = sorted(distances, reverse=True, key=lambda x: x[1])[:k]
            result_i.append((id1, k_smallest))
            
        result.append(result_i)
        
    result2 = []
    for x in result:
        result2_i = []
        for y in x:
            result2_i.append([z[0] for z in y[1]])
        result2.append(result2_i)
    
    return result2

In [15]:
def performance_accumulation_pixel(pixel_candidates, pixel_annotation):
    """ 
    performance_accumulation_pixel()

    Function to compute different performance indicators 
    (True Positive, False Positive, False Negative, True Negative) 
    at the pixel level
       
    [pixelTP, pixelFP, pixelFN, pixelTN] = performance_accumulation_pixel(pixel_candidates, pixel_annotation)
       
    Parameter name      Value
    --------------      -----
    'pixel_candidates'   Binary image marking the foreground areas
    'pixel_annotation'   Binary image containing ground truth
       
    The function returns the number of True Positive (pixelTP), False Positive (pixelFP), 
    False Negative (pixelFN) and True Negative (pixelTN) pixels in the image pixel_candidates
    """
    
    pixel_candidates = np.uint64(pixel_candidates>0)
    pixel_annotation = np.uint64(pixel_annotation>0)
    
    pixelTP = np.sum(pixel_candidates & pixel_annotation)
    pixelFP = np.sum(pixel_candidates & (pixel_annotation==0))
    pixelFN = np.sum((pixel_candidates==0) & pixel_annotation)
    pixelTN = np.sum((pixel_candidates==0) & (pixel_annotation==0))


    return [pixelTP, pixelFP, pixelFN, pixelTN]

In [16]:
# # Load ground truth files for each query
with open('C:/Users/Luis/Documents/Universidad/MCV/C1/Lab2/data/qsd2_w1/gt_corresps.pkl', 'rb') as f:
    gt_w1 = pickle.load(f)

with open('C:/Users/Luis/Documents/Universidad/MCV/C1/Lab2/data/qsd1_w2/gt_corresps.pkl', 'rb') as f:
    gt_w2_1 = pickle.load(f)

with open('C:/Users/Luis/Documents/Universidad/MCV/C1/Lab2/data/qsd2_w2/gt_corresps.pkl', 'rb') as f:
    gt_w2_2 = pickle.load(f)

## Validation results

### Method 1:
- Represent the original image in the HSV color space.
- Image descriptor: Concatenate the normalised histograms of the 3 image channels.
- Chi-square distance as the similarity function.

In [65]:
# Retrieval parameters
k = 1
sim_func = chi_squared_distance

for bins in [48]:
    for lvl in [0,2,4]:
        for d_hist in [1,2,3]:
            if lvl == 4 and d_hist == 3: continue
            # Create DataLoader objects for both the database and the queries
            data_loader = DataLoader('C:/Users/Luis/Documents/Universidad/MCV/C1/Lab2/data/BBDD')
            features_rgb, features_hsv, features_lab, _, _ = data_loader.load_data(remove_background=False, level=lvl, d_hist=d_hist, bins=bins)

            data_loader_q2 = DataLoader('data/qsd2_w1')
            features_rgb_q2_w1, features_hsv_q2_w1, features_lab_q2_w1, masks, masks_text = data_loader_q2.load_data(remove_background=True, remove_text=False, level=lvl, d_hist=d_hist, bins=bins)

            # Query 1: Results and mAP@k
            results_rgb_q1 = compare_images(features_rgb_q2_w1, features_rgb, k, sim_func)
            results_hsv_q1 = compare_images(features_hsv_q2_w1, features_hsv, k, sim_func)
            results_lab_q1 = compare_images(features_lab_q2_w1, features_lab, k, sim_func)
            
            mapk_rgb_1 = mapk(gt_w1, results_rgb_q1, 1)
            # mapk_rgb_5 = mapk(gt_w1, results_rgb_q1, 5)

            print(f'Bins: {bins}, Level: {lvl}, D_hist: {d_hist}, mAP@1: {mapk_rgb_1}')#, mAP@5: {mapk_rgb_5}')

            mapk_hsv_1 = mapk(gt_w1, results_hsv_q1, 1)
            # mapk_hsv_5 = mapk(gt_w1, results_hsv_q1, 5)

            print(f'Bins: {bins}, Level: {lvl}, D_hist: {d_hist}, mAP@1: {mapk_hsv_1}')#, mAP@5: {mapk_rgb_5}')

            mapk_lab_1 = mapk(gt_w1, results_lab_q1, 1)
            # mapk_lab_5 = mapk(gt_w1, results_lab_q1, 5)

            print(f'Bins: {bins}, Level: {lvl}, D_hist: {d_hist}, mAP@1: {mapk_lab_1}')#, mAP@5: {mapk_rgb_5}')

  0%|          | 0/287 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:04<00:00,  6.51it/s]


Bins: 48, Level: 0, D_hist: 1, mAP@1: 0.36666666666666664
Bins: 48, Level: 0, D_hist: 1, mAP@1: 0.4
Bins: 48, Level: 0, D_hist: 1, mAP@1: 0.4


100%|██████████| 30/30 [00:02<00:00, 10.08it/s]


Bins: 48, Level: 0, D_hist: 2, mAP@1: 0.43333333333333335
Bins: 48, Level: 0, D_hist: 2, mAP@1: 0.4666666666666667
Bins: 48, Level: 0, D_hist: 2, mAP@1: 0.3333333333333333


100%|██████████| 30/30 [00:18<00:00,  1.60it/s]


Bins: 48, Level: 0, D_hist: 3, mAP@1: 0.43333333333333335
Bins: 48, Level: 0, D_hist: 3, mAP@1: 0.43333333333333335
Bins: 48, Level: 0, D_hist: 3, mAP@1: 0.5


100%|██████████| 30/30 [00:04<00:00,  6.10it/s]


Bins: 48, Level: 2, D_hist: 1, mAP@1: 0.6
Bins: 48, Level: 2, D_hist: 1, mAP@1: 0.6
Bins: 48, Level: 2, D_hist: 1, mAP@1: 0.5333333333333333


 94%|█████████▍| 271/287 [01:39<00:02,  7.10it/s]

In [ ]:
# # Load ground truth files for each query
with open('data/qsd2_w2/text_boxes.pkl', 'rb') as f:
    boxes = pickle.load(f)

In [ ]:
# Query 2: Results and mAP@k
results_hsv_q2 = compare_images(features_hsv_q2, features_hsv, k, sim_func)

mapk_lab_1 = mapk(gt2, results_hsv_q2, 1)
mapk_lab_5 = mapk(gt2, results_hsv_q2, 5)
mapk_lab_10 = mapk(gt2, results_hsv_q2, 10)

mapk_lab_1, mapk_lab_5, mapk_lab_10

In [ ]:
# Query 2, removing background: Results and mAP@k
results_hsv_q2_no_back = compare_images(features_hsv_q2_no_back, features_hsv, k, sim_func)

mapk_lab_1 = mapk(gt2, results_hsv_q2_no_back, 1)
mapk_lab_5 = mapk(gt2, results_hsv_q2_no_back, 5)
mapk_lab_10 = mapk(gt2, results_hsv_q2_no_back, 10)

mapk_lab_1, mapk_lab_5, mapk_lab_10

### Method 2:
- Represent the original image in the CIE Lab color space.
- Image descriptor: Divide the image in 256 blocks, get the histogram for each one and concatenate all of them.
- Histogram intersection as the similarity function.

In [ ]:
# Retrieval parameters
k = 10
sim_func = histogram_intersection

# Create DataLoader objects for both the database and the queries
data_loader = DataLoader('BBDD')
_, _, features_lab, _ = data_loader.load_data(blocks=True, blockNumber=4)

data_loader_q1 = DataLoader('qsd1_w1')
_, _, features_lab_q1, _ = data_loader_q1.load_data(blocks=True, blockNumber=4)

data_loader_q2 = DataLoader('qsd2_w1')
# _, _, features_lab_q2, _ = data_loader_q2.load_data(blocks=True, blockNumber=256)
_, _, features_lab_q2_no_back, masks = data_loader_q2.load_data(blocks=True, blockNumber=4, remove_background=True)

In [ ]:
# Query 1: Results and mAP@k
results_lab_q1 = compare_images(features_lab_q1, features_lab, k, sim_func)

mapk_lab_1 = mapk(gt1, results_lab_q1, 1)
mapk_lab_5 = mapk(gt1, results_lab_q1, 5)
mapk_lab_10 = mapk(gt1, results_lab_q1, 10)

mapk_lab_1, mapk_lab_5, mapk_lab_10

In [ ]:
# Query 2: Results and mAP@k
results_lab_q2 = compare_images(features_lab_q2, features_lab, k, sim_func)

mapk_lab_1 = mapk(gt2, results_lab_q1, 1)
mapk_lab_5 = mapk(gt2, results_lab_q1, 5)
mapk_lab_10 = mapk(gt2, results_lab_q1, 10)

mapk_lab_1, mapk_lab_5, mapk_lab_10

In [ ]:
# Query 2, removing background: Results and mAP@k
results_lab_q2_no_back = compare_images(features_lab_q2_no_back, features_lab, k, sim_func)

mapk_lab_1 = mapk(gt2, results_lab_q2_no_back, 1)
mapk_lab_5 = mapk(gt2, results_lab_q2_no_back, 5)
mapk_lab_10 = mapk(gt2, results_lab_q2_no_back, 10)

mapk_lab_1, mapk_lab_5, mapk_lab_10

### Precision, Recall and F1-score of the masking method

In [ ]:
folder_path = "data/qsd2_w1"

precision, recall, f1 = [], [], []
for i, im_file in enumerate(glob.glob(os.path.join(folder_path, "*.png"))):
    mask_gt = cv2.imread(im_file, cv2.IMREAD_GRAYSCALE)
    pixelTP, pixelFP, pixelFN, pixelTN = performance_accumulation_pixel(masks_q[i][1], mask_gt)

    p = pixelTP / (pixelTP + pixelFP)
    r = pixelTP / (pixelTP + pixelFN)

    precision.append(p)
    recall.append(r)
    f1.append(2 * (p * r) / (p + r))

print("Precision:", np.mean(precision))
print("Recall:", np.mean(recall))
print("F1-score:", np.mean(f1))

In [ ]:
fig, ax = plt.subplots(5, 3, figsize=(10, 10))
for i in range(5):
    ax[i,0].imshow(masks_q[i][0], cmap='gray')
    ax[i,1].imshow(masks_q[i][1], cmap='gray')
    ax[i,2].imshow(masks_q[i][2], cmap='gray')

## Get test results from both queries

#### Method 1:

In [ ]:
# Create DataLoader objects for both the database and the queries
data_loader = DataLoader('data/BBDD')
_, features_hsv, _, _ = data_loader.load_data(blocks=False)

data_loader_q1 = DataLoader('data/qst1_w1')
_, features_hsv_q1, _, _ = data_loader_q1.load_data(blocks=False)

data_loader_q2_no_back = DataLoader('data/qst2_w1')
_, features_hsv_q2_no_back, _, masks = data_loader_q2_no_back.load_data(blocks=False, remove_background=True)

In [ ]:
# Retrieval parameters
k = 10
sim_func = chi_squared_distance

# Get results
results_hsv_q1 = compare_images(features_hsv_q1, features_hsv, k, sim_func)
results_hsv_q2_no_back = compare_images(features_hsv_q2_no_back, features_hsv, k, sim_func)

In [ ]:
# Save results as pickle files and masks as png images
with open("test_results/QST1/method1/results.pkl", "wb") as f:
    pickle.dump(results_hsv_q1, f)

with open("test_results/QST2/method1/results.pkl", "wb") as f:
    pickle.dump(results_hsv_q2_no_back, f)

for i, mask in enumerate(masks):
    final_mask = (mask[0]*mask[1]*mask[2])*255
    image_mask_name = str(i).zfill(5) + ".png"
    cv2.imwrite(os.path.join("test_results/QST2/method1/", image_mask_name), final_mask)

#### Method 2

In [ ]:
# Create DataLoader objects for both the database and the queries
data_loader = DataLoader('data/BBDD')
_, _, features_lab, _ = data_loader.load_data(blocks=True, blockNumber=256)

data_loader_q1 = DataLoader('data/qst1_w1')
_, _, features_lab_q1, _ = data_loader_q1.load_data(blocks=True, blockNumber=256)

data_loader_q2_no_back = DataLoader('data/qst2_w1')
_, _, features_lab_q2_no_back, masks = data_loader_q2_no_back.load_data(blocks=True, blockNumber=256, remove_background=True)

In [ ]:
# Retrieval parameters
k = 10
sim_func = histogram_intersection

# Get results
results_lab_q1 = compare_images(features_lab_q1, features_lab, k, sim_func)
results_lab_q2_no_back = compare_images(features_lab_q2_no_back, features_lab, k, sim_func)

In [ ]:
# Save result lists as pickle files and masks as png images
with open("test_results/QST1/method2/results.pkl", "wb") as f:
    pickle.dump(results_lab_q1, f)

with open("test_results/QST2/method2/results.pkl", "wb") as f:
    pickle.dump(results_lab_q2_no_back, f)

for i, mask in enumerate(masks):
    final_mask = (mask[0]*mask[1]*mask[2])*255
    image_mask_name = str(i).zfill(5) + ".png"
    cv2.imwrite(os.path.join("test_results/QST2/method2/", image_mask_name), final_mask)